<a href="https://colab.research.google.com/github/Chinemelu4/Customer-Prediction/blob/main/business_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files

In [ ]:
uploaded=files.upload()


Saving Audiobooks_data.csv to Audiobooks_data.csv


In [ ]:
import io
df=pd.read_csv(io.BytesIO(uploaded['Audiobooks_data.csv']))
df.head(2)

,00994,1620,1620.1,19.73,19.73.1,1,10.00,0.99,1603.80,5,92,0
0,1143,2160.0,2160,5.33,5.33,0,8.91,0.0,0.0,0,0,0
1,2059,2160.0,2160,5.33,5.33,0,8.91,0.0,0.0,0,388,0


In [ ]:
import numpy as np
from sklearn import preprocessing

In [ ]:
df=np.loadtxt('Audiobooks_data.csv',delimiter=',')

In [ ]:
unscaled_inputs=df[:,1:-1]
targets_all=df[:,-1]

In [ ]:
#Balance the targets my making sure we have the same number of zeros and ones for a balanced analysis

In [ ]:
num_one_targets=int(np.sum(targets_all))
zero_targets_counter=0
indices_to_remove=[]

for i in range (targets_all.shape[0]):
  if targets_all[i]==0:
    zero_targets_counter += 1
    if zero_targets_counter> num_one_targets:
      indices_to_remove.append(i)

unscaled_inputs_equal_priors=np.delete(unscaled_inputs, indices_to_remove, axis=0)
targets_equal_priors=np.delete(targets_all, indices_to_remove,axis=0)


scale all our inputs


In [ ]:
scaled_inputs=preprocessing.scale(unscaled_inputs_equal_priors)

shuffle data


In [ ]:
 shuffled_indices=np.arange(scaled_inputs.shape[0])
 np.random.shuffle(shuffled_indices)

 shuffled_inputs=scaled_inputs[shuffled_indices]
 shuffled_targets=targets_equal_priors[shuffled_indices]

In [ ]:
shuffled_inputs.shape

(4474, 10)

In [ ]:
samples_count=shuffled_inputs.shape[0]

train_samples_count=int(0.8*samples_count)
validation_samples_count=int(0.1*samples_count)
test_samples_count=samples_count - train_samples_count - validation_samples_count

train_inputs=shuffled_inputs[:train_samples_count]
train_targets=shuffled_targets[:train_samples_count]

validation_inputs=shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets=shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs=shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets=shuffled_targets[train_samples_count+validation_samples_count:]

print(test_inputs.shape)
print(validation_inputs.shape)
print(train_inputs.shape)

(448, 10)
(447, 10)
(3579, 10)


In [ ]:
print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets),validation_samples_count,np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count)

1779.0 3579 0.49706621961441744
249.0 447 0.5570469798657718
209.0 448 0.46651785714285715


In [ ]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs= validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets= test_targets)

In [ ]:
#create class for algorithm batching

In [ ]:
class Audiobooks_Data_Reader():
  def __init__(self, dataset, batch_size = None):
    npz = np.load('Audiobooks_data_{0}.npz'.format(dataset))
    self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
    if batch_size is None:
      self.batch_size = self.inputs.shape[0]
            
    else:
      self.batch_size = batch_size
    self.curr_batch = 0
    self.batch_count = self.inputs.shape[0] // self.batch_size
            
  def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1    
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        return inputs_batch, targets_one_hot
  def __iter__(self):
        return self        
        

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

TensorFlow 1.x selected.
Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
input_size=10
output_size=2
hidden_layer_size=200

#to rest any variables left in memory from previous runs (do this always so as totrain model from scratch when changing hyper-parameters)
tf.reset_default_graph()

In [ ]:
inputs=tf.placeholder(tf.float32,[None, input_size])
targets=tf.placeholder(tf.int32, [None, output_size])

In [ ]:
weights_1=tf.get_variable("weights_1", [input_size, hidden_layer_size])
biases_1= tf.get_variable("biases_1", [hidden_layer_size])

output_1=tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

weights_2= tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size])
biases_2= tf.get_variable("biases_2", [hidden_layer_size])

outputs_2= tf.nn.relu(tf.matmul(output_1, weights_2) + biases_2)

weights_3=tf.get_variable("weights_3", [hidden_layer_size, hidden_layer_size])
biases_3= tf.get_variable("biases_3", [hidden_layer_size])

outputs_3= tf.nn.relu(tf.matmul(outputs_2, weights_3) + biases_3)

weights_4=tf.get_variable("weights_4", [hidden_layer_size, output_size])
biases_4= tf.get_variable("biases_4", [output_size])


outputs= tf.matmul(outputs_3, weights_4) 

loss=tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=targets)

mean_loss=tf.reduce_mean(loss)
optimize=tf.train.AdamOptimizer(learning_rate=0.0035).minimize(mean_loss)

#check prediction accuracy
out_equals_targets=tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))
accuracy= tf.reduce_mean(tf.cast(out_equals_targets, tf.float32))


sess=tf.InteractiveSession()
initializer=tf.global_variables_initializer()
sess.run(initializer)


batch_size=100



max_epochs=50

prev_validation_loss= 9999999

train_data = Audiobooks_Data_Reader('train', batch_size)
validation_data = Audiobooks_Data_Reader('validation')
for epoch_counter in range(max_epochs):
  curr_epoch_loss=0.

  for input_batch, target_batch in train_data:
    _, batch_loss= sess.run([optimize, mean_loss],
        feed_dict={inputs: input_batch, targets: target_batch})
    curr_epoch_loss += batch_loss

  curr_epoch_loss/=train_data.batch_count

  validation_loss=0
  validation_accuracy=0

  for input_batch, target_batch in validation_data:
        validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],
            feed_dict={inputs: input_batch, targets: target_batch})
  curr_epoch_loss += batch_loss

  print('Epoch  '+str(epoch_counter+1)+
        '. Mean loss: '+'{0:.3f}'.format(curr_epoch_loss)+
        '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
        '. Vlidation accuracy:'+'{0:.2f}'.format(validation_accuracy + 100.)+'%')

  if validation_loss > prev_validation_loss:
    break

  prev_validation_loss=validation_loss

print ('End of Training') 








/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch  1. Mean loss: 0.868. Validation loss: 0.380. Vlidation accuracy:100.80%
Epoch  2. Mean loss: 0.780. Validation loss: 0.347. Vlidation accuracy:100.81%
Epoch  3. Mean loss: 0.755. Validation loss: 0.337. Vlidation accuracy:100.81%
Epoch  4. Mean loss: 0.719. Validation loss: 0.331. Vlidation accuracy:100.82%
Epoch  5. Mean loss: 0.694. Validation loss: 0.325. Vlidation accuracy:100.83%
Epoch  6. Mean loss: 0.677. Validation loss: 0.321. Vlidation accuracy:100.82%
Epoch  7. Mean loss: 0.675. Validation loss: 0.319. Vlidation accuracy:100.83%
Epoch  8. Mean loss: 0.678. Validation loss: 0.315. Vlidation accuracy:100.83%
Epoch  9. Mean loss: 0.687. Validation loss: 0.313. Vlidation accuracy:100.84%
Epoch  10. Mean loss: 0.674. Validation loss: 0.309. Vlidation accuracy:100.83%
Epoch  11. Mean loss: 0.663. Validation loss: 0.306. Vlidation accuracy:100.84%
Epoch  12. Mean loss: 0.654. Validation loss: 0.309. Vlidation accuracy:100.84%
End of Training


In [ ]:
test_data=Audiobooks_Data_Reader('test')
for input_batch, target_batch in test_data:
        test_accuracy = sess.run([accuracy],
            feed_dict={inputs: input_batch, targets: target_batch})

test_accuracy_percent= test_accuracy[0]*100    
print('Test accuracy: '+'{0:2f}'.format(test_accuracy_percent)+'%')

Test accuracy: 79.687500%
